# 参考
- [LangChain Python API Reference](https://python.langchain.com/api_reference/)

# [Build a Chatbot](https://python.langchain.com/docs/tutorials/chatbot/)

モデルは状態を持たないので、会話をしたいときは会話履歴全体をモデルに渡す必要があります。

ここでは [LangGraph](https://langchain-ai.github.io/langgraph/) の memory 機能を使って、会話の履歴を保持します。

## LangGraph

各LLMエージェントのステップなどをグラフ化して状態管理を行うためのツールです。

- [LangGraphの基本的な使い方](https://zenn.dev/pharmax/articles/8796b892eed183)
- [LangGraph](https://langchain-ai.github.io/langgraph/)
  - [MemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver)
  - [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph)


In [1]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = ChatOpenAI(model="gpt-4", temperature=0)

# 新しいGraphを定義
workflow = StateGraph(state_schema=MessagesState)

# モデルを呼び出す関数
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# graphにノードを追加
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# メモリ追加
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": "abc123"})

query = "わたしはボブです"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

こんにちは、ボブさん。何かお手伝いできることがありますか？


In [3]:
query = "私の名前は何ですか?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

あなたの名前はボブさんです。


In [4]:
# 別のスレッドで実行する
config = RunnableConfig(configurable={"thread_id": "asdf"})
query = "私の名前は何ですか？"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

申し訳ありませんが、私はあなたの名前を知りません。


In [5]:
# いつでも前のスレッドに戻れる
config = RunnableConfig(configurable={"thread_id": "abc123"})
query = "私の名前は何ですか?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

あなたの名前はボブさんです。


# プロンプトテンプレートを追加する